In [ ]:
# Standard library imports
import os
import sys
from pathlib import Path
from contextlib import redirect_stdout
import multiprocessing as mp
import datetime
import re
import pickle
import itertools

# Third-party imports
os.environ['USE_PYGEOS'] = '0'
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from matplotlib import patches as mpatches
from dateutil.relativedelta import relativedelta
from importlib import reload
from typing import Union, Dict, List, Tuple

from shapely.geometry import Point, Polygon
from shapely.ops import nearest_points

from sqlalchemy import create_engine

import pysal
import esda
import pysda
import libpysal as lps
from libpysal.weights.distance import get_points_array
from mgwr.gwr import GWR, MGWR
from mgwr.sel_bw import Sel_BW
from mgwr.utils import shift_colormap, truncate_colormap, compare_surfaces

import rasterio
from rasterio.plot import show_hist, show
from rasterio.mask import mask

from scipy.spatial import cKDTree

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score

import statsmodels.stats.outliers_influence as oi

import xgboost
import shap
from sklearn.model_selection import train_test_split
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, space_eval

from PIL import Image

from geofeather import to_geofeather, from_geofeather
from difflib import SequenceMatcher

import folium
from folium.plugins import Fullscreen

import contextily as ctx

# Local imports
sys.path.append('./utils')
import utils
import pyspace


# Configuration settings and setup code

pd.set_option('display.max_columns', 500)
pd.options.mode.chained_assignment = None  # default='warn'

In [ ]:
data_folder = Path('../data/')
results_folder = Path('../results/')

In [ ]:
cantons_ch = gpd.read_file(data_folder/'Administrative units'/'swissBOUNDARIES3D_1_3_TLM_KANTONSGEBIET.shp', engine='pyogrio')
cantons_ch = cantons_ch.to_crs(2056)
canton_vd = cantons_ch[(cantons_ch.NAME == 'Vaud')&(cantons_ch.EINWOHNERZ.isnull()==False)]

## Load cluster persistence and determinants

In [ ]:
df = gpd.read_parquet(data_folder/'df_survival_covariates.geoparquet')
df_period = gpd.read_parquet(data_folder/'df_survival_period_covariates.geoparquet')

df_lausanne = gpd.read_parquet(data_folder/'df_survival_covariates_lausanne.geoparquet')
df_period_lausanne = gpd.read_parquet(data_folder/'df_survival_period_covariates_lausanne.geoparquet')

## Process satellite imagery

In [ ]:
# Set up the data directory and file paths
pollution_folder = data_folder/"Pollution NO2/"
raster_file = "download.tropospheric_NO2_column_number_density 1.tif"
statpop_points_file = "statpop_points.feather"
statpop_ha_file = "statpop_communes_vd_ha.feather"

# Load the GeoDataFrames
statpop_points_gdf = from_geofeather(os.path.join(data_folder,'ag-b-00.03-vz2020statpop', statpop_points_file))
statpop_ha_gdf = from_geofeather(os.path.join(data_folder, 'ag-b-00.03-vz2020statpop', statpop_ha_file))

In [ ]:
output_file = "statpop_ha_with_NO2.feather"
output_path = os.path.join(pollution_folder, output_file)

if not os.path.exists(output_path):
    # Process all raster data to obtain min, max, mean, median NO2 values by month and for each hectare of the canton of VD
    all_stats = []
    try:
        for i in range(1, 45):
            raster_file = f"download.tropospheric_NO2_column_number_density {i}.tif"
            print(f"Processing raster {i}")
            
            stats, statpop_ha_gdf = utils.process_raster_data(
                raster_path=raster_file,
                index=i,
                data_dir=pollution_folder,
                statpop_points_gdf=statpop_points_gdf,
                statpop_ha_gdf=statpop_ha_gdf
            )
            all_stats.append(stats)
        
        print("All raster processing completed successfully.")
        
        # Convert all_stats to a DataFrame
        stats_df = pd.DataFrame(all_stats)
        stats_df.index += 1  # Make index start from 1
        
        print("\nStatistics summary for all processed rasters:")
        print(stats_df.describe())
        
        # Save the statistics
        stats_output_file = "raster_stats_summary.csv"
        stats_df.to_csv(os.path.join(pollution_folder, stats_output_file))
        print(f"\nRaster statistics saved to {stats_output_file}")
        
        # Save the updated GeoDataFrame
        to_geofeather(statpop_ha_gdf.filter(regex='RELI|geometry|NO2_tropospheric'), output_path)
        df_ha_no2 = from_geofeather(pollution_folder/'statpop_ha_with_NO2.feather')
        print(f"Updated GeoDataFrame saved to {output_file}")
    except FileNotFoundError as e:
        print(f"Error: {e}")
    except ValueError as e:
        print(f"Error: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
else:
    stats_output_file = "raster_stats_summary.csv"
    df_ha_no2 = from_geofeather(pollution_folder/'statpop_ha_with_NO2.feather')
    stats_df = pd.read_csv(os.path.join(pollution_folder, stats_output_file))
    print(f"'{output_file}' already exists. Skipping processing.")

In [ ]:
stats_no2_tropospheric = pd.read_csv(pollution_folder/'statisticalDataOfNO2.csv')
stats_no2_tropospheric  = stats_no2_tropospheric.round(4)

In [ ]:
stats_df = stats_df.round(4)

In [ ]:
correspondence_months_tropo = pd.merge(stats_df, stats_no2_tropospheric,how = 'left', right_on = 'tropospheric_NO2_column_number_density_max', left_on = 'max')
correspondence_months_tropo.index +=1
dict_months_tropo = correspondence_months_tropo['system:index'].to_dict() 


for key in dict_months_tropo.keys():
    df_ha_no2 = df_ha_no2.rename(columns = {'NO2_tropospheric_{}'.format(key):'NO2_tropospheric_{}'.format(dict_months_tropo[key])})

df_ha_no2['NO2_tropospheric-mean2019'] = df_ha_no2.filter(regex = 'NO2_tropospheric_2019').mean(axis = 1)
df_ha_no2['NO2_tropospheric-mean2020'] = df_ha_no2.filter(regex = 'NO2_tropospheric_2020').mean(axis = 1)
df_ha_no2['NO2_tropospheric-mean2021'] = df_ha_no2.filter(regex = 'NO2_tropospheric_2021').mean(axis = 1)
df_ha_no2['NO2_tropospheric-mean2022'] = df_ha_no2.filter(regex = 'NO2_tropospheric_2022').mean(axis = 1)

to_geofeather(df_ha_no2, pollution_folder/'df_ha_NO2_final.feather')

In [ ]:
df_no2_vd_month = df_ha_no2.filter(regex = 'NO2_tropospheric_|RELI').melt(id_vars = 'RELI')
df_no2_vd_year = df_ha_no2.filter(regex = 'NO2_tropospheric-mean|RELI').melt(id_vars = 'RELI')

In [ ]:
df_no2_vd_month['month'] = pd.to_datetime(df_no2_vd_month['variable'].str[-7:].str.replace('_',''), format = '%Y%m')
df_no2_vd_year['year'] = pd.to_datetime(df_no2_vd_year['variable'].str[-4:], format = '%Y')

### Merge with cluster persistence data

In [ ]:
df = pd.merge(df, df_ha_no2.filter(regex = 'RELI|NO2'), on = 'RELI')
df_period = pd.merge(df_period, df_ha_no2.filter(regex = 'RELI|NO2'), on = 'RELI')

df_lausanne = pd.merge(df_lausanne, df_ha_no2.filter(regex = 'RELI|NO2'), on = 'RELI')
df_period_lausanne = pd.merge(df_period_lausanne, df_ha_no2.filter(regex = 'RELI|NO2'), on = 'RELI')

## Final processing steps

In [ ]:
df_modelling = df[['RELI','Persistence_survival','LST','NDVI','pm25','pm10','no2','noise_car_night','index_socio_stand','B20BTOT','B20BTOT_lag8','B20BTOT_lag24','B20BTOT_lag200m','E_KOORD','N_KOORD','testing_rate','NO2_tropospheric-mean2020','NO2_tropospheric-mean2021','NO2_tropospheric-mean2022','geometry']].reset_index(drop = True)
df_modelling_lausanne = df_lausanne[['RELI','Persistence_survival','LST','NDVI','pm25','pm10','no2','noise_car_night','index_socio_stand','B20BTOT','B20BTOT_lag8','B20BTOT_lag24','B20BTOT_lag200m','E_KOORD','N_KOORD','testing_rate','NO2_tropospheric-mean2020','NO2_tropospheric-mean2021','NO2_tropospheric-mean2022','geometry']].reset_index(drop = True)

df_modelling_period = df_period[['RELI','Persistence_survival','LST','NDVI','pm25','pm10','no2','noise_car_night','index_socio_stand','B20BTOT','B20BTOT_lag8','B20BTOT_lag24','B20BTOT_lag200m','E_KOORD','N_KOORD','testing_rate','NO2_tropospheric-mean2020','NO2_tropospheric-mean2021','NO2_tropospheric-mean2022','period','geometry']].reset_index(drop = True)
df_modelling_period_lausanne = df_period_lausanne[['RELI','Persistence_survival','LST','NDVI','pm25','pm10','no2','noise_car_night','index_socio_stand','B20BTOT','B20BTOT_lag8','B20BTOT_lag24','B20BTOT_lag200m','E_KOORD','N_KOORD','testing_rate','NO2_tropospheric-mean2020','NO2_tropospheric-mean2021','NO2_tropospheric-mean2022','period','geometry']].reset_index(drop = True)

In [ ]:
dict_col_names = {'B20BTOT':'Population',
                 'B20BTOT_lag8':'Lagged population (8-NN)',
                 'B20BTOT_lag24':'Lagged population (24-NN)',
                 'B20BTOT_lag200m':'Lagged population (200m)',
                 'testing_rate':'Testing rate (%)',
                 'index_socio_stand':'SES index',
                 'LST':'Land Surface Temperature',
                 'NDVI':'NDVI',
                 'pm10':'PM10',
                'pm25':'PM25',
                 'no2':'NO2',
                 'noise_car_night':'Nighttime car noise',
                 'NO2_tropospheric-mean2020':'Tropospheric NO2 (2020)',
                 'NO2_tropospheric-mean2021':'Tropospheric NO2 (2021)',
                 'NO2_tropospheric-mean2022':'Tropospheric NO2 (2022)',
                 'E_KOORD':'E',
                 'N_KOORD':'N',
                 'period':'Period',
                 'geometry':'geometry'}

In [ ]:
# Rename columns
df_modelling = df_modelling.rename(columns=dict_col_names)
df_modelling_period = df_modelling_period.rename(columns=dict_col_names)

df_modelling_lausanne = df_modelling_lausanne.rename(columns=dict_col_names)
df_modelling_period_lausanne = df_modelling_period_lausanne.rename(columns=dict_col_names)

In [ ]:
df_modelling['Tropospheric NO2'] = df_modelling.filter(regex = "Tropospheric").mean(axis = 1)
df_modelling_period['Tropospheric NO2'] = df_modelling_period.filter(regex = "Tropospheric").mean(axis = 1)

df_modelling_lausanne['Tropospheric NO2'] = df_modelling_lausanne.filter(regex = "Tropospheric").mean(axis = 1)
df_modelling_period_lausanne['Tropospheric NO2'] = df_modelling_period_lausanne.filter(regex = "Tropospheric").mean(axis = 1)

## Modelling functions

In [ ]:
# Choose hyperparameter domain to search over
space = {
        'max_depth':hp.choice('max_depth', np.arange(1, 30, 2, dtype=int)),
        'colsample_bytree':hp.quniform('colsample_bytree', 0.3, 1.01, 0.1),
        'min_child_weight':hp.choice('min_child_weight', np.arange(1, 30, 1, dtype=int)),
        'subsample':        hp.quniform('subsample', 0.3, 1.01, 0.1),
        'learning_rate':    hp.choice('learning_rate',    np.arange(0.1, 1.01, 0.1)),
        'gamma': hp.quniform('gamma', 0.1, 5, 0.2),
    
        'objective':'reg:squarederror',
        'eval_metric': 'rmse',
    }

In [ ]:
def train_xgboost(df, X_eq, y_col, optimize, space, n_evals, file_prefix, include_spatial=True):
    model_directory = results_folder / 'Modelling' / file_prefix
    model_directory.mkdir(parents=True, exist_ok=True)

    X_coords = df[X_eq + ['E', 'N']] if include_spatial else df[X_eq]
    y = df[y_col]
    X_train, X_test, y_train, y_test = train_test_split(X_coords, y, test_size=0.2, random_state=0)

    d_train = xgboost.DMatrix(X_train, label=y_train)
    d_test = xgboost.DMatrix(X_test, label=y_test)
    d_all = xgboost.DMatrix(X_coords, label=y)

    # Hyperparameter tuning
    best_params_path = model_directory / 'best_params.pkl'
    if not best_params_path.exists():
        trials = Trials()
        best_params = optimize(trials, space, n_evals=n_evals)
        best_params = space_eval(space, best_params)
        with open(model_directory / 'trials.pkl', 'wb') as f:
            pickle.dump(trials, f)
        with open(best_params_path, 'wb') as f:
            pickle.dump(best_params, f)
    else:
        with open(best_params_path, 'rb') as f:
            best_params = pickle.load(f)
        with open(model_directory / 'trials.pkl', 'rb') as f:
            trials = pickle.load(f)

    # Train model
    model_path = model_directory / 'model.pkl'
    if not model_path.exists():
        final_model = xgboost.train(
            best_params, d_train, num_boost_round=500,
            evals=[(d_train, "train"), (d_test, "test")],
            verbose_eval=False, early_stopping_rounds=10
        )
        with open(model_path, 'wb') as f:
            pickle.dump(final_model, f)
    else:
        with open(model_path, 'rb') as f:
            final_model = pickle.load(f)

    # Evaluate model on test set
    y_pred_test = final_model.predict(d_test)
    print("Test set performance:")
    print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_test)))
    print("R2:", r2_score(y_test, y_pred_test))

    # Evaluate model on full dataset
    y_pred_all = final_model.predict(d_all)
    print("Full dataset performance:")
    print('RMSE:', np.sqrt(mean_squared_error(y, y_pred_all)))
    print('R2: ', r2_score(y, y_pred_all))

    interpretable_ml_shap_viz(df, final_model, X_coords, model_directory, include_spatial)
    
    return final_model

In [ ]:
def score(params, n_folds=5):
    # If you want to sample the data !
    # _X_coords_sample = X_coords.sample(frac = 0.2, random_state=333).sort_index()
    # _y_sample = y[y.index.isin(_X_coords_sample.index)]
    #Cross-validation
    d_train = xgboost.DMatrix(X_coords, y)
    
    cv_results = xgboost.cv(params, d_train, nfold = n_folds, num_boost_round=500,
                        early_stopping_rounds = 10, metrics = 'rmse', seed = 0)
    
    loss = min(cv_results['test-rmse-mean'])
    
    return loss


def optimize(trials, space, n_evals):
    
    best = fmin(score, space, algo=tpe.suggest, max_evals=n_evals,trials=trials,
                rstate=np.random.default_rng(333))#Add seed to fmin function
    return best


In [ ]:
def interpretable_ml_shap_viz(df, model, X, model_directory: Union[str, Path], include_spatial: bool = True):
    """
    Generate and save various SHAP (SHapley Additive exPlanations) visualizations for an XGBoost model.

    Args:
        df: DataFrame containing the data, including spatial information if include_spatial is True.
        model: Trained XGBoost model.
        X: Feature matrix used for training the model.
        model_directory (Union[str, Path]): Directory to save the generated visualizations.
        include_spatial (bool): Whether to include spatial visualizations. Defaults to True.

    Raises:
        ValueError: If the model directory cannot be created.
        Exception: For any other unexpected errors during execution.
    """
    # try:
    model_directory = Path(model_directory)
    model_directory.mkdir(parents=True, exist_ok=True)

    explainer_shap = shap.TreeExplainer(model)
    shap_values = explainer_shap(X)
    shap_interaction_values = explainer_shap.shap_interaction_values(X)

    generate_shap_summary(shap_values, X, model_directory)
    generate_shap_interaction_summary(shap_interaction_values, X, model_directory)
    generate_dependence_plots(shap_values, X, model_directory)

    if include_spatial:
        generate_shap_maps(df, shap_values, X, model_directory)
        generate_location_effect_map(df, shap_values, model_directory)

    generate_variable_importance_plots(model, model_directory)
    generate_force_plot(explainer_shap, shap_values, X, model_directory)

    # except ValueError as ve:
    #     print(f"Error creating model directory: {ve}")
    # except Exception as e:
    #     print(f"An unexpected error occurred: {e}")

def generate_shap_summary(shap_values, X, model_directory):
    shap.summary_plot(shap_values, feature_names=X.columns, show=False)
    save_plot(model_directory, 'shap_summary')
    plt.close()

def generate_shap_interaction_summary(shap_interaction_values, X, model_directory):
    shap.summary_plot(shap_interaction_values, X, max_display=16,
                      feature_names=X.columns, show=False, plot_type="compact_dot")
    save_plot(model_directory, 'shap_summary_interaction_values')
    plt.close()

def generate_dependence_plots(shap_values, X, model_directory):
    dependence_dir = model_directory / 'Dependence plots'
    dependence_dir.mkdir(exist_ok=True)

    for name in X.columns:
        shap.dependence_plot(name, shap_values.values, X, display_features=X, alpha=0.4, interaction_index=None, show=False)
        save_plot(dependence_dir, f'shap_dependence_{name}', formats=['png'])
        plt.close()

def generate_shap_maps(df, shap_values, X, model_directory):
    n_columns = len(X.columns)
    n_rows = int(np.ceil(n_columns / 3))
    fig, ax = plt.subplots(n_rows, 3, figsize=(15, 15 * n_rows / 3))
    ax = ax.ravel()

    for j in range(n_columns):
        df.plot(ax=ax[j], column=shap_values.values[:, j], legend=True,
                vmin=-0.8, vmax=0.8, cmap=shap.plots.colors.red_white_blue, legend_kwds={'shrink':0.5})
        ax[j].set_title("SHAP for\n" + str(X.columns[j]), fontsize=10)
        ax[j].set_axis_off()

    for j in range(n_columns, n_rows * 3):
        ax[j].set_axis_off()

    plt.tight_layout()
    save_plot(model_directory, 'maps_shap_variables')
    plt.close()
def generate_location_effect_map(df, shap_values, model_directory):
    fig, ax = plt.subplots(dpi=300)
    df.plot(ax=ax, column=shap_values.values[:, -1] + shap_values.values[:, -2],
            legend=True, vmin=-0.6, vmax=0.6, figsize=(15, 8),
            cmap=shap.plots.colors.red_white_blue)
    plt.title("Location Effect on Cluster persistence\n(SHAP values of geographic coordinates)\n", fontsize=8)
    plt.axis('off')
    save_plot(model_directory, 'maps_shap_location')
    plt.close()

def generate_variable_importance_plots(model, model_directory):
    xgboost.plot_importance(model)
    plt.title("xgboost.plot_importance(model)")
    save_plot(model_directory, 'importance_plot_default')

    xgboost.plot_importance(model, importance_type="cover")
    plt.title('xgboost.plot_importance(model, importance_type="cover")')
    save_plot(model_directory, 'importance_plot_cover')
    plt.close()

def generate_force_plot(explainer_shap, shap_values, X, model_directory):
    shap.force_plot(explainer_shap.expected_value, shap_values.values[1, :], X.iloc[1, :], show=False, matplotlib=True)
    save_plot(model_directory, 'force_plot')
    plt.close()

def save_plot(directory, filename, formats=['png']):
    for fmt in formats:
        plt.savefig(directory / f'{filename}.{fmt}', dpi=300 if fmt == 'png' else 180, bbox_inches='tight')
    plt.close()

## Run XGBoost + SHAP

### Whole canton
#### Whole period
##### Univariate analyses


In [ ]:
y = df['Persistence_survival']
X_eq0 = ['Population']
X_eq1 = ['Lagged population (8-NN)']
X_eq2 = ['Lagged population (200m)']
X_eq3 = ['Lagged population (24-NN)']
X_eq4 = ['Testing rate (%)']
X_eq5 = ['SES index']
X_eq6 = ['Land Surface Temperature']
X_eq7 = ['NDVI']
X_eq8 = ['PM10']
X_eq9 = ['NO2']
X_eq10 = ['Tropospheric NO2']
X_eq11 = ['Nighttime car noise']
X_eq12 = ['E']
X_eq13 = ['N']
X_eq14 = ['PM25']

In [ ]:
feature_sets = [X_eq0, X_eq1, X_eq2, X_eq3, X_eq4, X_eq5, X_eq6, X_eq7, X_eq8, X_eq9, X_eq10, X_eq11, X_eq12, X_eq13, X_eq14]

for i, features in enumerate(feature_sets):
    print(features)
    train_xgboost(df_modelling, features, 'Persistence_survival', optimize, space, n_evals=500, file_prefix=f'Model {i} - Univariate', include_spatial=False)

##### Multivariate analyses

In [ ]:
y = df_modelling['Persistence_survival']

X_eq1 = ['Population','Lagged population (24-NN)','SES index','Testing rate (%)']
X_eq2 = ['Population','Lagged population (24-NN)','SES index','Land Surface Temperature','NDVI','PM25','NO2','Nighttime car noise','Tropospheric NO2','Testing rate (%)']

X_eq3 = ['Population','Lagged population (24-NN)','PM25','Testing rate (%)']
X_eq4 = ['Population','Lagged population (24-NN)','PM10','Testing rate (%)']
X_eq7 = ['Population','Lagged population (24-NN)','NO2','Testing rate (%)']
X_eq8 = ['Population','Lagged population (24-NN)','Tropospheric NO2','Testing rate (%)']

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Calculating VIF for each feature
X_coords = df_modelling[X_eq3]
vif_data = pd.DataFrame()
vif_data["feature"] = X_coords.columns
vif_data["VIF"] = [variance_inflation_factor(X_coords.values, i) for i in range(X_coords.shape[1])]

print("VIF values:")
print(vif_data)

In [ ]:
# train_xgboost(df_modelling, X_eq1, 'Persistence_survival', optimize, space, n_evals=500, file_prefix='Model 1')
# train_xgboost(df_modelling, X_eq2, 'Persistence_survival', optimize, space, n_evals=500, file_prefix='Model 2')

In [ ]:
# train_xgboost(df_modelling, X_eq3, 'Persistence_survival', optimize, space, n_evals=500, file_prefix='Model 3')
# train_xgboost(df_modelling, X_eq4, 'Persistence_survival', optimize, space, n_evals=500, file_prefix='Model 4')
# train_xgboost(df_modelling, X_eq5, 'Persistence_survival', optimize, space, n_evals=500, file_prefix='Model 5')
# train_xgboost(df_modelling, X_eq6, 'Persistence_survival', optimize, space, n_evals=500, file_prefix='Model 6')

#### Period-wise analysis
##### Multivariate analysis

In [ ]:
variables = ['RELI', 'Land Surface Temperature', 'NDVI', 'SES index', 'PM10','PM25', 'NO2', 'Nighttime car noise', 'Population', 'Lagged population (8-NN)', 'Lagged population (24-NN)','Lagged population (200m)', 'Tropospheric NO2 (2020)', 'Tropospheric NO2 (2021)', 'Tropospheric NO2 (2022)', 'Testing rate (%)']
periods = ['first_period', 'second_period', 'third_period', 'fourth_period', 'fifth_period']
dfs = []
for period in periods:
    df_period = df_modelling_period[df_modelling_period['Period'] == period]
    df_period.loc[df_period['NO2'] > df_period['NO2'].quantile(0.999), 'NO2'] = df_period['NO2'].mean()
    df_period.loc[df_period['PM10'] > df_period['PM10'].quantile(0.999), 'PM10'] = df_period['PM10'].mean()
    df_period = pd.merge(df_period, df_ha_no2.filter(regex='RELI|NO2'), on='RELI')
    df_period = gpd.GeoDataFrame(df_period, geometry=df_period['geometry'], crs=2056)

    dfs.append(df_period)

df_period_1, df_period_2, df_period_3, df_period_4, df_period_5 = dfs
df_period_1['Tropospheric NO2 (periodic avg)'] = df_period_1[['NO2_tropospheric_2020_3','NO2_tropospheric_2020_4','NO2_tropospheric_2020_5','NO2_tropospheric_2020_6']].mean( axis = 1)
df_period_2['Tropospheric NO2 (periodic avg)'] = df_period_2[['NO2_tropospheric_2020_7','NO2_tropospheric_2020_8','NO2_tropospheric_2020_9','NO2_tropospheric_2020_10','NO2_tropospheric_2020_11','NO2_tropospheric_2020_12']].mean( axis = 1)
df_period_3['Tropospheric NO2 (periodic avg)'] = df_period_3[['NO2_tropospheric_2021_1','NO2_tropospheric_2021_2','NO2_tropospheric_2021_3','NO2_tropospheric_2021_4','NO2_tropospheric_2021_5']].mean( axis = 1)
df_period_4['Tropospheric NO2 (periodic avg)'] = df_period_4[['NO2_tropospheric_2021_6','NO2_tropospheric_2021_7','NO2_tropospheric_2021_8','NO2_tropospheric_2021_9','NO2_tropospheric_2021_10', 'NO2_tropospheric_2021_11']].mean( axis = 1)
df_period_5['Tropospheric NO2 (periodic avg)'] = df_period_5[['NO2_tropospheric_2021_12','NO2_tropospheric_2022_1','NO2_tropospheric_2022_2','NO2_tropospheric_2022_3','NO2_tropospheric_2022_4']].mean( axis = 1)

##### Dimensionality reduction 

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler

def apply_pca_to_df(df, feature_names, new_feature_name, inverted=False):
    # Selecting the relevant features
    X = df[feature_names]

    # Standardizing the features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Applying PCA
    pca = PCA(n_components=1)
    X_pca = pca.fit_transform(X_scaled)

    # # Scaling PCA results to range 0-100
    # scaler_100 = MinMaxScaler(feature_range=(0, 100))
    # X_pca_scaled = scaler_100.fit_transform(X_pca)

    df[new_feature_name] = X_pca[:, 0]
    if inverted:
        # Inverting the scale
        X_pca_inverted = 100 - X_pca
        # Adding the inverted scaled PCA result back to the dataframe
        df[new_feature_name] = X_pca_inverted[:, 0]

In [ ]:
# List of features to combine into a single PCA component
features_to_combine = ['Land Surface Temperature', 'NDVI', 'Nighttime car noise']
new_feature_name = 'Urban type index'

# Applying PCA to each DataFrame
dataframes = [df_period_1, df_period_2, df_period_3, df_period_4, df_period_5]
for i, df in enumerate(dataframes, start=1):
    dataframes[i-1] = apply_pca_to_df(df, features_to_combine, new_feature_name, inverted=False)
    print(f"PCA applied to gwr_df_vd_period_{i}")

In [ ]:
# List of features to combine into a single PCA component
features_to_combine = ['NO2','PM25','PM10']
new_feature_name = 'Air pollution index'

# Applying PCA to each DataFrame
dataframes = [df_period_1, df_period_2, df_period_3, df_period_4, df_period_5]
for i, df in enumerate(dataframes, start=1):
    dataframes[i-1] = apply_pca_to_df(df, features_to_combine, new_feature_name)
    print(f"PCA applied to gwr_df_vd_period_{i}")

##### Multicollinearity check

In [ ]:
import pandas as pd
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Assuming df_period_1, df_period_2, ..., df_period_5 are already defined

periods = ['first', 'second', 'third', 'fourth', 'fifth']
dfs = [df_period_1, df_period_2, df_period_3, df_period_4, df_period_5]

features = ['Population', 'Lagged population (24-NN)', 'Air pollution index', 
            'Tropospheric NO2 (periodic avg)', 'Testing rate (%)', 'Urban type index']

for period, df in zip(periods, dfs):
    print(f"\nCalculating VIF for {period} period:")
    
    # Selecting features for VIF calculation
    X_coords = df[features]
    
    # Calculating VIF for each feature
    vif_data = pd.DataFrame()
    vif_data["feature"] = X_coords.columns
    vif_data["VIF"] = [variance_inflation_factor(X_coords.values, i) 
                       for i in range(X_coords.shape[1])]
    
    # Sorting VIF values in descending order
    vif_data = vif_data.sort_values('VIF', ascending=False)
    
    print(vif_data)
    
    # Optional: You can add a threshold check here
    high_vif = vif_data[vif_data['VIF'] > 5]  # Assuming 5 as a threshold
    if not high_vif.empty:
        print(f"\nFeatures with high VIF (>5) in {period} period:")
        print(high_vif)
    else:
        print(f"\nNo features with high VIF (>5) in {period} period.")

##### Run models

In [ ]:
X_eq1_wc = ['Population','Lagged population (24-NN)','Air pollution index','Tropospheric NO2 (periodic avg)','Testing rate (%)', 'Urban type index']
X_eq2_wc = ['Population','Lagged population (24-NN)','SES index','Testing rate (%)', 'Urban type index']

In [ ]:
periods = [df_period_1, df_period_2, df_period_3, df_period_4, df_period_5]

for i, period in enumerate(periods, start=1):
    y = period['Persistence_survival']
    train_xgboost(period, X_eq1_wc, 'Persistence_survival', optimize, space, n_evals=500, file_prefix=f'Model 1 - Period {i}')
    train_xgboost(period, X_eq2_wc, 'Persistence_survival', optimize, space, n_evals=500, file_prefix=f'Model 2 - Period {i}')

### Lausanne urban area
#### Whole period

In [ ]:
df_modelling_lausanne[['RELI','NO2']][df_modelling_lausanne['RELI'] == 53821550]

In [ ]:
X_eq1 = ['Population','Lagged population (24-NN)','SES index','Testing rate (%)']
X_eq2 = ['Population','Lagged population (24-NN)','SES index','Land Surface Temperature','NDVI','PM25','NO2','Nighttime car noise','Tropospheric NO2','Testing rate (%)']

X_eq3 = ['Population','Lagged population (24-NN)','PM25','Testing rate (%)']
X_eq4 = ['Population','Lagged population (24-NN)','PM10','Testing rate (%)']
X_eq5 = ['Population','Lagged population (24-NN)','NO2','Testing rate (%)']
X_eq6 = ['Population','Lagged population (24-NN)','Tropospheric NO2','Testing rate (%)']

In [ ]:
train_xgboost(df_modelling_lausanne, X_eq1, 'Persistence_survival', optimize, space, n_evals=500, file_prefix='Model 1 - Lausanne urban area')
train_xgboost(df_modelling_lausanne, X_eq2, 'Persistence_survival', optimize, space, n_evals=500, file_prefix='Model 2 - Lausanne urban area')
train_xgboost(df_modelling_lausanne, X_eq3, 'Persistence_survival', optimize, space, n_evals=500, file_prefix='Model 3 - Lausanne urban area')
train_xgboost(df_modelling_lausanne, X_eq4, 'Persistence_survival', optimize, space, n_evals=500, file_prefix='Model 4 - Lausanne urban area')
train_xgboost(df_modelling_lausanne, X_eq5, 'Persistence_survival', optimize, space, n_evals=500, file_prefix='Model 5 - Lausanne urban area')
train_xgboost(df_modelling_lausanne, X_eq6, 'Persistence_survival', optimize, space, n_evals=500, file_prefix='Model 6 - Lausanne urban area')